## YOLOv8 – Road Damage

In [1]:
from ultralytics import YOLO

model = YOLO("runs/detect/train4/weights/best.pt")

metrics = model.val(
    data="/home/imadb/.cache/kagglehub/datasets/alvarobasily/road-damage/versions/road_damage_yolo/dataset/dataset.yaml",
    imgsz=640,
    device=0
)

print("mAP50-95 :", metrics.box.map)
print("mAP50    :", metrics.box.map50)
print("Precision:", metrics.box.p)
print("Recall   :", metrics.box.r)


Ultralytics 8.4.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
Model summary (fused): 72 layers, 3,006,428 parameters, 0 gradients, 8.1 GFLOPs


FileNotFoundError: '/home/imadb/.cache/kagglehub/datasets/alvarobasily/road-damage/versions/road_damage_yolo/dataset/dataset.yaml' does not exist

### Commentaire sur les résultats (YOLOv8 – Road Damage)

Le modèle YOLOv8 obtient de **bons résultats** pour la détection des dégâts routiers.  
Le **mAP@50 = 0.76** montre qu’il détecte correctement la majorité des défauts, tandis que le **mAP@50–95 = 0.45** indique que la localisation des boîtes est correcte mais peut encore être améliorée, surtout pour les fissures fines.  
La **précision (0.77)** signifie qu’il y a relativement peu de fausses détections, et le **rappel (0.71)** montre que quelques défauts peuvent encore être manqués.  
Les classes **pothole** et **alligator_cracking** sont bien détectées, alors que **lateral_cracking** reste la plus difficile.


In [20]:
import pandas as pd

results_path = "/home/imadb/Ghiles/Projet_Ghiles/bdd_100k/runs/detect/train_micro3/results.csv"

df = pd.read_csv(results_path)

# Dernière époque (celle du best.pt)
last = df.iloc[-1]

print("Precision :", last["metrics/precision(B)"])
print("Recall    :", last["metrics/recall(B)"])
print("mAP50     :", last["metrics/mAP50(B)"])
print("mAP50-95  :", last["metrics/mAP50-95(B)"])


Precision : 0.50663
Recall    : 0.11766
mAP50     : 0.1027
mAP50-95  : 0.05373


## Résultats du modèle YOLOv8 sur BDD100K

Les performances du modèle entraîné sur le dataset **BDD100K** sont résumées ci-dessous :

- Précision (Precision) : 0.51  
- Rappel (Recall) : 0.12  
- mAP@50 : 0.10  
- mAP@50–95 : 0.05  

### Analyse des métriques

La précision relativement modérée (0.51) indique qu’environ une détection sur deux est correcte, ce qui montre que le modèle ne produit pas excessivement de faux positifs.  
En revanche, le rappel très faible (0.12) signifie que le modèle détecte seulement une petite fraction des objets présents dans les images, manquant ainsi la majorité des cibles.

Le score mAP@50 de 0.10 révèle une capacité limitée à détecter correctement les objets, même avec un seuil d’IoU relativement permissif.  
Le score mAP@50–95, encore plus bas (0.05), met en évidence une faible précision dans la localisation des boîtes englobantes lorsque des critères plus stricts sont appliqués.

### Interprétation globale

Ces résultats montrent que, dans sa configuration actuelle, le modèle présente des performances faibles sur le dataset BDD100K.  
Ce jeu de données est particulièrement complexe en raison de la diversité des scènes, de la présence d’objets de petite taille et d’un fort déséquilibre entre les classes. Comparativement au dataset Road Damage, BDD100K constitue un défi nettement plus difficile et nécessite un entraînement plus approfondi ou un modèle plus robuste pour obtenir de meilleures performances.


In [22]:
from ultralytics import YOLO
import os

model_path = "/home/imadb/Ghiles/Projet_Ghiles/bdd_100k/runs/detect/train_micro3/weights/best.pt"
data_yaml  = "/home/imadb/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/bdd100k_yolo_10cls/bdd100k_10cls.yaml"

print("Model existe ?", os.path.exists(model_path))
print("YAML existe ?", os.path.exists(data_yaml))

model = YOLO(model_path)

metrics = model.val(
    data=data_yaml,
    imgsz=640,
    device=0
)

print("mAP50-95 :", metrics.box.map)
print("mAP50    :", metrics.box.map50)
print("Precision:", metrics.box.p)
print("Recall   :", metrics.box.r)


Model existe ? True
YAML existe ? True
Ultralytics 8.4.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
Model summary (fused): 72 layers, 3,007,598 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1002.0±186.7 MB/s, size: 51.0 KB)
val: Scanning /home/imadb/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/bdd100k_yolo_10cls/labels/val... 10000 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 10000/10000 564.4it/s 17.7s.0s
val: New cache created: /home/imadb/.cache/kagglehub/datasets/solesensei/solesensei_bdd100k/versions/2/bdd100k_yolo_10cls/labels/val.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 625/625 15.2it/s 41.1s<0.1s
                   all      10000     185517      0.526      0.165      0.147     0.0756
                person       3220      13261      0.344      0.386       0.32      0.139
                 rider        515       

In [2]:
from ultralytics import YOLO
from pathlib import Path
import yaml

candidates = [
    ("Obstacle_LAF",  "/home/imadb/Ghiles/runs/detect/train4/weights/best.pt"),
    ("RoadWorks",     "/home/imadb/runs/detect/RoadWorks2/weights/best.pt"),
    ("SpeedBumps",    "/home/imadb/runs/detect/SpeedBumps/weights/best.pt"),
    ("StopSign",      "/home/imadb/runs/detect/StopSign/weights/best.pt"),
    ("RoadDamage",    "/home/imadb/Ghiles/Projet_Ghiles/runs/detect/train4/weights/best.pt"),
    ("RoadDebris1",   "/home/imadb/runs/detect/RoadDebris1/weights/best.pt"),
    ("RoadObstacle",  "/home/imadb/runs/detect/road_obstacle_yolov8l/weights/best.pt"),
]

def find_data_yaml_from_run(weights_path: Path):
    run_dir = weights_path.parent.parent  # .../weights/best.pt -> run_dir
    args_yaml = run_dir / "args.yaml"
    if not args_yaml.exists():
        return None, f"args.yaml not found: {args_yaml}"
    try:
        data = yaml.safe_load(args_yaml.read_text())
    except Exception as e:
        return None, f"cannot read args.yaml: {e}"
    data_path = data.get("data", None)
    if not data_path:
        return None, "data field not found in args.yaml"
    return Path(str(data_path)), None

print("\n=============== METRICS SUMMARY ================")

for name, w in candidates:
    w = Path(w)
    if not w.exists():
        print(f"[SKIP] {name} weights not found: {w}")
        continue

    data_yaml, err = find_data_yaml_from_run(w)
    if err:
        print(f"[SKIP] {name} {err}")
        continue
    if not data_yaml.exists():
        print(f"[SKIP] {name} data.yaml not found: {data_yaml}")
        continue

    model = YOLO(str(w))
    metrics = model.val(
        data=str(data_yaml),
        device=0,
        imgsz=640,
        conf=0.25,
        iou=0.70,
        verbose=False
    )

    print(
        f"{name:<14} | "
        f"P={metrics.box.mp:.3f} | "
        f"R={metrics.box.mr:.3f} | "
        f"mAP50={metrics.box.map50:.3f} | "
        f"mAP50-95={metrics.box.map:.3f} | "
        f"data={data_yaml}"
    )

print("=================================================\n")



=============== METRICS SUMMARY ================
[SKIP] Obstacle_LAF data.yaml not found: lost_and_found_od_yolo/data.yaml
Ultralytics 8.4.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
Model summary (fused): 113 layers, 43,608,150 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 161.3±37.1 MB/s, size: 41.7 KB)
val: Scanning /home/imadb/Ghiles/road work sing 1/road works sign/val/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 11.7Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 2.7it/s 1.5s0.6s
                   all         50        125      0.988      0.989      0.992      0.971
Speed: 2.4ms preprocess, 11.1ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /home/imadb/Ghiles/Projet_Ghiles/runs/detect/val10
RoadWorks      | P=0.988 | R=0.989 | mAP50=0.992 | mAP50-95=0.971 | data=/home/imadb/Ghile

In [3]:
import pandas as pd
from pathlib import Path

runs = [
    ("Obstacle_LAF",  "/home/imadb/Ghiles/runs/detect/train4"),
    ("RoadWorks",     "/home/imadb/runs/detect/RoadWorks2"),
    ("SpeedBumps",    "/home/imadb/runs/detect/SpeedBumps"),
    ("StopSign",      "/home/imadb/runs/detect/StopSign"),
    ("RoadDamage",    "/home/imadb/Ghiles/Projet_Ghiles/runs/detect/train4"),
    ("RoadDebris1",   "/home/imadb/runs/detect/RoadDebris1"),
    ("RoadObstacle",  "/home/imadb/runs/detect/road_obstacle_yolov8l"),
]

print("\n=============== METRICS FROM TRAINING ===============\n")

for name, run_dir in runs:
    run_dir = Path(run_dir)
    csv = run_dir / "results.csv"

    if not csv.exists():
        print(f"[SKIP] {name} results.csv not found")
        continue

    df = pd.read_csv(csv)

    last = df.iloc[-1]  # dernière epoch

    print(
        f"{name:<14} | "
        f"P={last['metrics/precision(B)']:.3f} | "
        f"R={last['metrics/recall(B)']:.3f} | "
        f"mAP50={last['metrics/mAP50(B)']:.3f} | "
        f"mAP50-95={last['metrics/mAP50-95(B)']:.3f}"
    )

print("\n====================================================\n")



=============== METRICS FROM TRAINING ===============

Obstacle_LAF   | P=0.994 | R=0.969 | mAP50=0.982 | mAP50-95=0.723
RoadWorks      | P=0.993 | R=0.981 | mAP50=0.988 | mAP50-95=0.960
SpeedBumps     | P=0.917 | R=0.863 | mAP50=0.883 | mAP50-95=0.493
StopSign       | P=0.957 | R=0.977 | mAP50=0.989 | mAP50-95=0.854
RoadDamage     | P=0.629 | R=0.546 | mAP50=0.576 | mAP50-95=0.264
RoadDebris1    | P=0.558 | R=0.667 | mAP50=0.570 | mAP50-95=0.283
RoadObstacle   | P=0.814 | R=0.727 | mAP50=0.747 | mAP50-95=0.407




In [4]:
from ultralytics import YOLO
from pathlib import Path
import yaml

candidates = [
    ("RoadWorks",     "/home/imadb/runs/detect/RoadWorks2/weights/best.pt"),
    ("SpeedBumps",    "/home/imadb/runs/detect/SpeedBumps/weights/best.pt"),
    ("StopSign",      "/home/imadb/runs/detect/StopSign/weights/best.pt"),
    ("RoadDebris1",   "/home/imadb/runs/detect/RoadDebris1/weights/best.pt"),
    ("RoadObstacle",  "/home/imadb/runs/detect/road_obstacle_yolov8l/weights/best.pt"),
]

def load_args(weights_path: Path):
    run_dir = weights_path.parent.parent
    args_yaml = run_dir / "args.yaml"
    if not args_yaml.exists():
        return None, run_dir
    return yaml.safe_load(args_yaml.read_text()), run_dir

print("=============== METRICS (REVAL SAME AS TRAIN) ===============")
for name, w in candidates:
    w = Path(w)
    if not w.exists():
        print(f"[SKIP] {name} weights not found")
        continue

    args, run_dir = load_args(w)
    if args is None:
        print(f"[SKIP] {name} args.yaml not found in {run_dir}")
        continue

    data = Path(str(args.get("data", "")))
    if not data.is_absolute():
        data = (run_dir / data).resolve()

    if not data.exists():
        print(f"[SKIP] {name} data.yaml not found: {data}")
        continue

    imgsz = args.get("imgsz", 640)
    conf  = args.get("conf", 0.001)   # training val often uses low conf internally
    iou   = args.get("iou", 0.7)

    model = YOLO(str(w))
    m = model.val(data=str(data), imgsz=imgsz, conf=conf, iou=iou, device=0, verbose=False)

    print(f"{name:<12} | P={m.box.mp:.3f} | R={m.box.mr:.3f} | mAP50={m.box.map50:.3f} | mAP50-95={m.box.map:.3f}")
print("=============================================================")



=============== METRICS (REVAL SAME AS TRAIN) ===============
Ultralytics 8.4.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48570MiB)
Model summary (fused): 113 layers, 43,608,150 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1385.5±437.0 MB/s, size: 38.6 KB)
val: Scanning /home/imadb/Ghiles/road work sing 1/road works sign/val/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 50/50 3.9Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 3.0it/s 1.3s0.6s
                   all         50        125      0.982      0.984      0.989      0.964
Speed: 3.7ms preprocess, 7.8ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /home/imadb/Ghiles/Projet_Ghiles/runs/detect/val15
RoadWorks    | P=0.982 | R=0.984 | mAP50=0.989 | mAP50-95=0.964
Ultralytics 8.4.2 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA RTX A6000, 48570MiB)